In [2]:
import json
import pandas as pd
from collections import Counter
from statistics import mean
import numpy as np

In [2]:
df = pd.read_json("imdb_dataset_parsed.json")

In [3]:
df.head(3)

,actors,directors,genre,metascore,rating,runtime,title,users_rating,votes,year
0,"[Chris Hemsworth, Michael Shannon, Michael Peñ...",[Nicolai Fuglsig],"[Action, Drama, History, War]",54.0,R,130 min,12 Strong,6.6,"42,919",2018.0
1,"[Jodie Foster, Lawrence A. Bonney, Kasi Lemmon...",[Jonathan Demme],"[Crime, Drama, Thriller]",85.0,R,118 min,The Silence of the Lambs,8.6,"1,091,846",1991.0
2,"[Harrison Ford, Rutger Hauer, Sean Young, Edwa...",[Ridley Scott],"[Sci-Fi, Thriller]",89.0,R,117 min,Blade Runner,8.2,"609,925",1982.0


In [4]:
df = df.dropna()

In [5]:
# ottenimento lista dei direttori per la creazione del dizionario
directors_list = []
for directors in df["directors"]:
    for director in directors:
        directors_list.append(director)
directors_set = set(directors_list)
directors_list = list(directors_set)

In [7]:
# creazione del dizionario
my_dict = dict((el,[]) for el in directors_list)
for index,row in df.iterrows():
    for director in row["directors"]:
        my_dict[director].append(row["actors"])

In [8]:
# ottenimento di un secondo dizionario contnente, per ogni direttore, 
# la lista di tutti gli attori che hanno partecipato nei suoi film
final_dict = {}
for k,v in my_dict.items():
    final_dict[k] = [item for sublist in v for item in sublist]

In [9]:
# selezione degli attori che hanno lavorato di più con un certo direttore
for k in final_dict.keys():
    final_dict[k] = Counter(final_dict[k]).most_common(1)

In [10]:
# ottenimento del metascore dei film relativo alla coppia direttore,regista
for index,row in df.iterrows():
    for director in row["directors"]:
        for actor in row["actors"]:
            if(actor in final_dict[director][0][0]):
                final_dict[director].append(row["metascore"])

In [11]:
# creazione del dataset finale, contente coppie di attori registi che hanno lavorato almeno in quattro film
new_dict = {}
for k,v in final_dict.items():
    try:
        if final_dict[k][0][1] > 4:
            new_dict[k] = [final_dict[k][0][0],final_dict[k][0][1], final_dict[k][1:]]
    except:
        continue
df_couples = pd.DataFrame.from_dict(new_dict, orient='index')
df_couples.reset_index(level=0, inplace=True)
df_couples.columns = ["director", "actor", "number_of_films", "metascore_mean"]

In [13]:
# ottenimento della media dei metascore
mean_scores = []
for index,row in df_couples.iterrows():
    mean_scores.append(mean(row["metascore_mean"]))

df_couples["metascore_mean"] = mean_scores

In [14]:
df_couples.head()

In [26]:
# ottenimento e salvataggio in due file diversi di coppie registi-attori che risultano essere "diversi" o la stessa persona

actor_is_director = df_couples['actor'] == df_couples['director']
df_couples_same = df_couples[actor_is_director].sort_values(['number_of_films'], ascending = False)
df_couples_same

,director,actor,number_of_films,metascore_mean
48,Clint Eastwood,Clint Eastwood,20,65.200000
40,Woody Allen,Woody Allen,18,69.055556
0,Sylvester Stallone,Sylvester Stallone,7,52.142857
17,Spike Lee,Spike Lee,7,65.857143
20,Tyler Perry,Tyler Perry,7,35.142857
37,James Franco,James Franco,6,43.166667
44,Ben Stiller,Ben Stiller,6,57.166667
51,Edward Burns,Edward Burns,6,54.000000
38,Albert Brooks,Albert Brooks,5,65.000000
52,Danny DeVito,Danny DeVito,5,59.000000


In [31]:
df_couples_different = df_couples[~actor_is_director].sort_values(['number_of_films'], ascending = False)
df_couples_different

,director,actor,number_of_films,metascore_mean
15,Garry Marshall,Hector Elizondo,8,46.875000
28,Blake Edwards,Graham Stark,8,54.500000
8,Wes Anderson,Bill Murray,8,78.500000
24,Dennis Dugan,Adam Sandler,8,33.500000
22,Tim Burton,Johnny Depp,8,69.375000
39,Martin Scorsese,Robert De Niro,8,81.125000
53,Robert Rodriguez,Danny Trejo,7,56.714286
6,Sam Raimi,Ted Raimi,7,64.000000
32,Richard Donner,Mel Gibson,6,54.333333
29,Richard Linklater,Ethan Hawke,6,81.500000


In [32]:
df_couples_same.to_json("couples_same.json", orient = 'records')
df_couples_different.to_json("couples_different.json", orient = 'records')

In [1]:
# unione colonne direttori-attori per maggior facilità di rappresentazione

In [5]:
df_different = pd.read_json("couples_same.json")

In [6]:
df_different.head()

,director,actor,number_of_films,metascore_mean
0,Clint Eastwood,Clint Eastwood,20,65.200000
1,Woody Allen,Woody Allen,18,69.055556
2,Sylvester Stallone,Sylvester Stallone,7,52.142857
3,Spike Lee,Spike Lee,7,65.857143
4,Tyler Perry,Tyler Perry,7,35.142857


In [15]:
df_different["couples"] = df_different["director"].map(str) + ", " + df_different["actor"]


In [17]:
df_different.to_json("couples_different.json", orient = 'records')

In [ ]:
df_same = pd.read_json("couples_same.json")